In [19]:
# this NOTEBOOK has the connection to S3 to read and write

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np

## Data Connection To S3

In [2]:
#Import packages for data connection to S3
import boto3
from io import StringIO

In [3]:
# AWS credentials
# Download the credentials in teams and save in your local machine, update the path as needed
file_path = 'C:/churn/bucketcredentials.csv'
credentials_df = pd.read_csv(file_path)

aws_access_key_id = credentials_df['aws_access_key_id'].iloc[0]
aws_secret_access_key = credentials_df['aws_secret_access_key'].iloc[0]
bucket_name = credentials_df['bucket_name'].iloc[0]

# for testing purposes you can change the path so you will not overwrite others output file
target_subfolder = "prescila" # change this accordingly

input_file_key = f"TargetDataSet/{target_subfolder}/bank_data_test.csv"
output_file_key = f"TargetDataSet/{target_subfolder}/model3.csv"

In [4]:
# Create an S3 client
s3_client = boto3.client('s3', aws_access_key_id=aws_access_key_id, aws_secret_access_key=aws_secret_access_key)

In [14]:
# Read the CSV file from S3 into a DataFrame
obj = s3_client.get_object(Bucket=bucket_name, Key=input_file_key)
X = pd.read_csv(obj['Body'])
X_train=X

In [25]:
# Read in the data in local machine
#df = pd.read_csv('bank_data_train.csv')
# This is replaced using S3 bucket

In [15]:
X_train.head(2)

,ID,CR_PROD_CNT_IL,AMOUNT_RUB_CLO_PRC,PRC_ACCEPTS_A_EMAIL_LINK,APP_REGISTR_RGN_CODE,PRC_ACCEPTS_A_POS,PRC_ACCEPTS_A_TK,TURNOVER_DYNAMIC_IL_1M,CNT_TRAN_AUT_TENDENCY1M,SUM_TRAN_AUT_TENDENCY1M,...,REST_DYNAMIC_CC_3M,MED_DEBT_PRC_YWZ,LDEAL_ACT_DAYS_PCT_TR3,LDEAL_ACT_DAYS_PCT_AAVG,LDEAL_DELINQ_PER_MAXYWZ,TURNOVER_DYNAMIC_CC_3M,LDEAL_ACT_DAYS_PCT_TR,LDEAL_ACT_DAYS_PCT_TR4,LDEAL_ACT_DAYS_PCT_CURR,TARGET
0,146841,0,0.000000,NaN,NaN,NaN,NaN,0.0,NaN,NaN,...,0.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0
1,146842,0,0.041033,NaN,NaN,NaN,NaN,0.0,0.166667,0.186107,...,0.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0


Start of K code

In [16]:
useless_cols = [col for col in X_train.columns if len(pd.Series(X_train[col].unique()).dropna()) <= 1]
useless_cols = useless_cols
print(useless_cols)
X_train.drop(columns=useless_cols, inplace=True)

['PRC_ACCEPTS_A_EMAIL_LINK', 'PRC_ACCEPTS_A_POS', 'PRC_ACCEPTS_A_TK', 'PRC_ACCEPTS_A_AMOBILE', 'PRC_ACCEPTS_TK', 'PRC_ACCEPTS_A_MTP', 'CNT_ACCEPTS_TK', 'PRC_ACCEPTS_A_ATM', 'PRC_ACCEPTS_MTP', 'CNT_ACCEPTS_MTP']


In [18]:
X_train.info

<bound method DataFrame.info of             ID  CR_PROD_CNT_IL  AMOUNT_RUB_CLO_PRC  APP_REGISTR_RGN_CODE  \
0       146841               0            0.000000                   NaN   
1       146842               0            0.041033                   NaN   
2       146843               0            0.006915                   NaN   
3       146844               0            0.000000                   NaN   
4       146845               0            0.000000                   NaN   
...        ...             ...                 ...                   ...   
355185  590822               0            0.000000                   NaN   
355186  590823               0            0.000000                   NaN   
355187  590825               0            0.041298                   NaN   
355188  590826               0            0.000000                   NaN   
355189  590828               0            0.000000                   NaN   

        TURNOVER_DYNAMIC_IL_1M  CNT_TRAN_AUT_TENDENCY1M

In [9]:
X_train.describe(include='all')

,ID,CR_PROD_CNT_IL,AMOUNT_RUB_CLO_PRC,APP_REGISTR_RGN_CODE,TURNOVER_DYNAMIC_IL_1M,CNT_TRAN_AUT_TENDENCY1M,SUM_TRAN_AUT_TENDENCY1M,AMOUNT_RUB_SUP_PRC,SUM_TRAN_AUT_TENDENCY3M,CLNT_TRUST_RELATION,...,REST_DYNAMIC_CC_3M,MED_DEBT_PRC_YWZ,LDEAL_ACT_DAYS_PCT_TR3,LDEAL_ACT_DAYS_PCT_AAVG,LDEAL_DELINQ_PER_MAXYWZ,TURNOVER_DYNAMIC_CC_3M,LDEAL_ACT_DAYS_PCT_TR,LDEAL_ACT_DAYS_PCT_TR4,LDEAL_ACT_DAYS_PCT_CURR,TARGET
count,355190.000000,355190.000000,316867.000000,60550.000000,355190.000000,77112.000000,77112.000000,316867.000000,111052.000000,69421,...,355190.000000,95713.000000,93448.000000,98175.000000,95713.000000,355190.000000,93448.000000,93448.000000,93448.000000,355190.000000
unique,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,FRIEND,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,24896,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,368794.674875,0.105225,0.044045,50.947498,0.001305,0.416896,0.414572,0.085249,0.689080,NaN,...,0.007309,0.055074,0.025707,0.049943,0.009252,0.004309,0.013938,0.013938,0.013938,0.081435
std,128148.804566,0.431372,0.108449,21.777855,0.029118,0.316493,0.338612,0.142310,0.301725,NaN,...,0.066681,0.215909,0.115732,0.185830,0.092789,0.059852,0.097099,0.097099,0.097099,0.273503
min,146841.000000,0.000000,0.000000,0.000000,0.000000,0.006944,0.000000,0.000000,0.000002,NaN,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,257846.250000,0.000000,0.000000,33.000000,0.000000,0.166667,0.139645,0.000000,0.446269,NaN,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,368778.500000,0.000000,0.000000,54.000000,0.000000,0.300000,0.285714,0.027117,0.722985,NaN,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,479737.750000,0.000000,0.036608,72.000000,0.000000,0.571429,0.661195,0.110005,1.000000,NaN,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [20]:
# Drop columns that only have one unique value
useless_cols = [col for col in X_train.columns if len(pd.Series(X_train[col].unique()).dropna()) <= 1]
useless_cols = useless_cols
print(useless_cols)
X_train.drop(columns=useless_cols, inplace=True)

[]


In [11]:
X_train.describe(include='all')

,ID,CR_PROD_CNT_IL,AMOUNT_RUB_CLO_PRC,APP_REGISTR_RGN_CODE,TURNOVER_DYNAMIC_IL_1M,CNT_TRAN_AUT_TENDENCY1M,SUM_TRAN_AUT_TENDENCY1M,AMOUNT_RUB_SUP_PRC,SUM_TRAN_AUT_TENDENCY3M,CLNT_TRUST_RELATION,...,REST_DYNAMIC_CC_3M,MED_DEBT_PRC_YWZ,LDEAL_ACT_DAYS_PCT_TR3,LDEAL_ACT_DAYS_PCT_AAVG,LDEAL_DELINQ_PER_MAXYWZ,TURNOVER_DYNAMIC_CC_3M,LDEAL_ACT_DAYS_PCT_TR,LDEAL_ACT_DAYS_PCT_TR4,LDEAL_ACT_DAYS_PCT_CURR,TARGET
count,355190.000000,355190.000000,316867.000000,60550.000000,355190.000000,77112.000000,77112.000000,316867.000000,111052.000000,69421,...,355190.000000,95713.000000,93448.000000,98175.000000,95713.000000,355190.000000,93448.000000,93448.000000,93448.000000,355190.000000
unique,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,FRIEND,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,24896,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,368794.674875,0.105225,0.044045,50.947498,0.001305,0.416896,0.414572,0.085249,0.689080,NaN,...,0.007309,0.055074,0.025707,0.049943,0.009252,0.004309,0.013938,0.013938,0.013938,0.081435
std,128148.804566,0.431372,0.108449,21.777855,0.029118,0.316493,0.338612,0.142310,0.301725,NaN,...,0.066681,0.215909,0.115732,0.185830,0.092789,0.059852,0.097099,0.097099,0.097099,0.273503
min,146841.000000,0.000000,0.000000,0.000000,0.000000,0.006944,0.000000,0.000000,0.000002,NaN,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,257846.250000,0.000000,0.000000,33.000000,0.000000,0.166667,0.139645,0.000000,0.446269,NaN,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,368778.500000,0.000000,0.000000,54.000000,0.000000,0.300000,0.285714,0.027117,0.722985,NaN,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,479737.750000,0.000000,0.036608,72.000000,0.000000,0.571429,0.661195,0.110005,1.000000,NaN,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [21]:
X_train.info

<bound method DataFrame.info of             ID  CR_PROD_CNT_IL  AMOUNT_RUB_CLO_PRC  APP_REGISTR_RGN_CODE  \
0       146841               0            0.000000                   NaN   
1       146842               0            0.041033                   NaN   
2       146843               0            0.006915                   NaN   
3       146844               0            0.000000                   NaN   
4       146845               0            0.000000                   NaN   
...        ...             ...                 ...                   ...   
355185  590822               0            0.000000                   NaN   
355186  590823               0            0.000000                   NaN   
355187  590825               0            0.041298                   NaN   
355188  590826               0            0.000000                   NaN   
355189  590828               0            0.000000                   NaN   

        TURNOVER_DYNAMIC_IL_1M  CNT_TRAN_AUT_TENDENCY1M

In [22]:
X_train_categorical = X_train.select_dtypes(include=["object_"])
categorical_cols = X_train_categorical.columns
print(categorical_cols)
X_train.drop(columns=categorical_cols, inplace=True)
X_train_categorical = X_train_categorical.apply(lambda c: c.str.lower())

In [23]:
ru_to_en = {'друг': 'other', 'мать': 'mother', 'брат': 'brother', 'отец': 'father', 'сестра': 'sister', 'дочь': 'daughter', 'сын': 'son', 'мама': 'mother'}
X_train_categorical['CLNT_TRUST_RELATION'].replace(ru_to_en, inplace=True)
X_train_categorical.describe()

C:\Users\presc\AppData\Local\Temp\ipykernel_7624\3514819626.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  X_train_categorical['CLNT_TRUST_RELATION'].replace(ru_to_en, inplace=True)


,CLNT_TRUST_RELATION,APP_MARITAL_STATUS,APP_KIND_OF_PROP_HABITATION,CLNT_JOB_POSITION_TYPE,CLNT_JOB_POSITION,APP_DRIVING_LICENSE,APP_EDUCATION,APP_TRAVEL_PASS,APP_CAR,APP_POSITION_TYPE,APP_EMP_TYPE,APP_COMP_TYPE,PACK
count,69421,68234,59361,44781,210811,57257,68104,57257,57256,60545,67362,67362,355190
unique,13,8,5,4,15075,2,12,2,2,4,4,4,12
top,friend,m,so,specialist,директор,n,h,n,n,specialist,private,private,102
freq,24896,32185,28056,25123,24974,36332,44370,52750,32843,36622,59087,59087,116986
